# Implementation of Using Convolution Filters with clustering techniques (DBSCAN) and Understanding of CNN via layer wise relevance propagation (or similar techniques) 

In [38]:
%time

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.preprocessing import normalize

plt.rcParams['figure.figsize'] = 5.0, 4.0

from pyts.transformation import GADF,GASF
from sklearn.preprocessing import normalize

import uproot
import torch
from torch import nn


CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 9.06 µs


### Definitions

In [2]:
def standard(x,height,decay):
    y = height*np.exp(-x*decay)
    return y

def shifter(x,starts):
    L = int(starts*len(x))
    y = np.zeros(len(x))
    y[:L] = np.zeros(L) 
    y[L:] = x[:(len(x)-L)]
    return y

def comb_standard(x,second,height_1,decay_1,height_2,decay_2):
    L = int(second*len(x))
    y = np.zeros(len(x))
    y[:L] = standard(x[:L],height_1,decay_1)
    y[L:] = standard(x[:(len(x)-L)],height_2,decay_2)
    return y

def noiser(x,strength):
    y = x + np.random.normal(0,strength,len(x))
    return y

def noiser_long(x,strength):
    noise = np.random.normal(0,strength,len(x))
    y = x + np.cumsum(noise)*strength
    return y

def noiser_comb(x,sepfact,strength):
    L = int(sepfact*len(x))
    x[:L] = noiser(x[:L],strength)
    x[L:] = noiser_long(x[L:],strength)
    return x

def array_maker(entries):
    x = np.arange(0,1,1/4096)
    x = np.expand_dims(x,axis=0)
    x = np.tile(x,[entries,1])
    return x

def double(x,second,height_1,decay_1,height_2,decay_2,starts,sepfact=0.15,strength=0.02):
    y = comb_standard(x,second,height_1,decay_1,height_2,decay_2)
    y = shifter(y,starts)
    y = noiser_comb(y,sepfact,strength)
    return y

def single(x,height,decay,starts,sepfact=0.15,strength=0.02):
    y = standard(x,height,decay)
    y = shifter(y,starts)
    y = noiser_comb(y,sepfact,strength)
    return y

def event_creators_single(entries):
    x = array_maker(entries)
    for i in range(entries):
        w = np.random.normal(1,0.01)
        r = np.random.normal(5,2)
        s = np.random.normal(0.03,0.005)
        x[i] = single(x[i],w,r,s)
    return x

def event_creators_single_2(entries):
    x = array_maker(entries)
    for i in range(entries):
        w = np.random.normal(1,0.01)
        r = np.random.normal(10,2)
        s = np.random.normal(0.03,0.005)
        x[i] = single(x[i],w,r,s)
    return x

def event_creators_single_3(entries):
    x = array_maker(entries)
    for i in range(entries):
        w = np.random.normal(1,0.1)
        r = np.random.normal(4,1)
        s = np.random.normal(0.03,0.005)
        x[i] = single(x[i],w,r,s)
    return x

def event_creators_sharp(entries):
    x = array_maker(entries)
    for i in range(entries):
        w = np.random.normal(1,0.01)
        r = np.random.normal(100,20)
        s = np.random.normal(0.03,0.005)
        x[i] = single(x[i],w,r,s)    
    return x

def event_creators_double_equal(entries):
    x = array_maker(entries)
    for i in range(entries):
        w = np.random.normal(0.2,0.01)
        p = np.random.normal(1,0.01)
        q = np.random.normal(5,2)
        r = np.random.normal(1,0.01)
        s = np.random.normal(5,2)
        t = np.random.normal(0.03,0.005)
        x[i] = double(x[i],w,p,q,r,s,t)
    return x
    
def event_creators_double_unequal(entries):
    x = array_maker(entries)
    for i in range(entries):
        w = np.random.normal(0.2,0.01)
        p = np.random.normal(1,0.2)
        q = np.random.normal(50,10)
        r = np.random.normal(1,0.2)
        s = np.random.normal(50,10)
        t = np.random.normal(0.03,0.005)
        x[i] = double(x[i],w,p,q,r,s,t)
    return x

def event_creators_sharp_fat(entries):
    x = array_maker(entries)
    for i in range(entries):
        w = np.random.normal(0.015,0.001)
        p = np.random.normal(1,0.2)
        q = np.random.normal(100,20)
        r = np.random.normal(0.2,0.05)
        s = np.random.normal(5,2)
        t = np.random.normal(0.03,0.005)
        x[i] = double(x[i],w,p,q,r,s,t)
    return x

def label(q,k):
    x = np.zeros(len(q))
    for i in range(len(q)):
        x[i] = k
    return x

def sep(q,k,z):
    y = label(q,k)
    x1, x2 ,x3 = np.split(q,[int(len(q)*training_ratio),int(len(q)*(training_ratio+validation_ratio))])
    y1, y2 ,y3 = np.split(y,[int(len(q)*training_ratio),int(len(q)*(training_ratio+validation_ratio))])
    if z == 0:
        return x1, y1
    if z == 1:
        return x2, y2
    if z == 2:
        return x3, y3

def reader_pmtall(path):
    extra = np.arange(4096, 4480)
    tree = uproot.open(path)["tree"]
    pmtall = tree.array("PMTALL")
    pmtall = np.delete(pmtall, extra, axis=1)
    return pmtall

def reader(path,branch,number):
    tree = uproot.open(path)["tree"]
    column = tree.array(branch)
    column = column[:,number]
    return column

def reader_lone(path,branch):
    tree = uproot.open(path)["tree"]
    column = tree.array(branch)
    return column

def pmtall_pedestal(path):
    pedestal = reader(path,"Pedestal",0)
    pmtall = reader_pmtall(path)
    for i in range(len(pedestal)):
        pmtall[i] = -(pmtall[i]-pedestal[i])
    
    return pmtall

In [12]:
def comb(one,two,three,four,five,six,seven,eight,nine,ten,portion):
    one1,one2 = sep(one,1,portion)
    two1,two2 = sep(two,1,portion)
    three1,three2 = sep(three,1,portion)
    four1,four2 = sep(four,1,portion)
    five1,five2 = sep(five,1,portion)
    six1,six2 = sep(six,0,portion)
    seven1,seven2 = sep(seven,0,portion)
    eight1,eight2 = sep(eight,0,portion)
    nine1,nine2 = sep(nine,0,portion)
    ten1,ten2 = sep(ten,0,portion)

    z = np.concatenate((one1,two1,three1,four1,five1,six1,seven1,eight1,nine1,ten1),axis=0)
    y = np.concatenate((one2,two2,three2,four2,five2,six2,seven2,eight2,nine2,ten2),axis=0)
    return z, y

# Load data
build it using python class

In [200]:
%time
class Waveform():
    
    def __init__(self, path=None, no_classes=None):
        if path is None:
            raise ValueError("Insert file path!")
        if no_classes is None:
            raise ValueError("Number of classes?")
        
        # Load PMTALL(sum of waveform of CANDLES), removing last portion of data
        tree = uproot.open(path)["tree"]
        extra = np.arange(4096,4480)
        pmtall = tree.array("PMTALL")
        pmtall = np.delete(pmtall, extra, axis=1)
        pedestal = tree.array("Pedestal")
        pedestal_sum = pedestal[:,0]
        for i in range(len(pedestal_sum)):
            pmtall[i] = -(pmtall[i]-pedestal_sum[i])
#         number = 
        
        # random labelling(test purposes)
        self.waveform = pmtall
        self.label = np.random.randint(3,size=(len(pmtall),))
    
    def __len__(self):
        return self.waveform.shape[0]
    
    def __getitem__(self,idx):
        return (self.waveform[idx],self.label[idx])


CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 8.82 µs


In [201]:
no_classes = 3
dataset = Waveform(path="Run009-069-001.root", no_classes=no_classes)

In [202]:
BATCH_SIZE = 1000
from torch.utils.data import DataLoader
data_loader = DataLoader(dataset=dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=True,
                         num_workers=3) 

# #Testing#
# for i in dataset:
#     print(len(i[0]),len(i[1]))
# for i in data_loader:
#     print(i[0].size())
#     print(i[1].size())
# print(len(dataset))


In [222]:
n_batches = int(len(dataset)/BATCH_SIZE)

In [223]:
n_batches

41

# Define CNN structure
using an autoencoder for self-training, taking encoder part or decoder part for features learning

In [149]:
# Discriminator
Discriminator = nn.Sequential(
    nn.Linear(4096, 512),
    nn.LeakyReLU(0.2),
    nn.Linear(512, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 64),
    nn.LeakyReLU(0.2),
    nn.Linear(64, 64),
    nn.LeakyReLU(0.2),
    nn.Linear(64, no_classes),
    nn.Sigmoid()
)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(Discriminator.parameters(), lr=0.001, momentum=0.9)

print(Discriminator)
for parameter in Discriminator.parameters():
    print(parameter.size())

Sequential(
  (0): Linear(in_features=4096, out_features=512, bias=True)
  (1): LeakyReLU(0.2)
  (2): Linear(in_features=512, out_features=256, bias=True)
  (3): LeakyReLU(0.2)
  (4): Linear(in_features=256, out_features=64, bias=True)
  (5): LeakyReLU(0.2)
  (6): Linear(in_features=64, out_features=64, bias=True)
  (7): LeakyReLU(0.2)
  (8): Linear(in_features=64, out_features=3, bias=True)
  (9): Sigmoid()
)
torch.Size([512, 4096])
torch.Size([512])
torch.Size([256, 512])
torch.Size([256])
torch.Size([64, 256])
torch.Size([64])
torch.Size([64, 64])
torch.Size([64])
torch.Size([3, 64])
torch.Size([3])


# training
transform to Torch.Variable

In [150]:
from torch.autograd import Variable

def to_var(x):
    # first move to GPU, if necessary
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)

# Dummy code workspace

In [185]:
output = Variable(torch.randn(10,120))
target = Variable(torch.FloatTensor(10).uniform_(0, 120).long())
print(torch.FloatTensor(10))
print(torch.FloatTensor(10).uniform_(0,1).long())
# print(output,target)

loss = criterion(output,target)
print(loss)


 0.0000e+00
 0.0000e+00
 1.9945e-28
-3.6893e+19
 4.2039e-45
 9.8091e-45
 4.3489e-25
 1.4013e-45
 0.0000e+00
 0.0000e+00
[torch.FloatTensor of size 10]


 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
[torch.LongTensor of size 10]

Variable containing:
 5.0597
[torch.FloatTensor of size 1]



In [231]:
N_EPOCHS = 10

# allow for manual keyboard interrupt
try: 
    # loop through epochs
    for epoch in range(N_EPOCHS):
        for batch_number, (waveform,label) in enumerate(data_loader):
            
#             print("epoch=",epoch)
#             print(batch_number)
#             print(waveform.size(),label.size())
    
            batch_size = waveform.size()[0]
            training_data = to_var(waveform.view(batch_size,-1))
            target = to_var(label.view(batch_size,-1))
            
#             print(training_data,training_label)
            
            outputs = Discriminator(training_data)
#             print(outputs, target.view(-1).long())
            
            real_score = outputs
            loss = criterion(outputs, target.view(-1).long())
            loss.backward()
            optimizer.step()
#             print(loss.data[0])
        
            if (batch_number +1)%5 == 0:
                print("Epoch[%d/%d], Step[%d/%d], loss=%.4f, Mean Discriminator output=%.4f"
                      %(epoch,
                        N_EPOCHS,
                        batch_number+1,
                        n_batches,
                        loss.data[0],output.data.mean()))
        

except KeyboardInterrupt:
    print('Training ended early.')



Epoch[0/10], Step[5/41], loss=1.0991, Mean Discriminator output=-0.0129
Epoch[0/10], Step[10/41], loss=1.0986, Mean Discriminator output=-0.0129
Epoch[0/10], Step[15/41], loss=1.0991, Mean Discriminator output=-0.0129
Epoch[0/10], Step[20/41], loss=1.0981, Mean Discriminator output=-0.0129
Epoch[0/10], Step[25/41], loss=1.0991, Mean Discriminator output=-0.0129
Epoch[0/10], Step[30/41], loss=1.0981, Mean Discriminator output=-0.0129
Epoch[0/10], Step[35/41], loss=1.0986, Mean Discriminator output=-0.0129
Epoch[0/10], Step[40/41], loss=1.0990, Mean Discriminator output=-0.0129
Epoch[1/10], Step[5/41], loss=1.0981, Mean Discriminator output=-0.0129
Epoch[1/10], Step[10/41], loss=1.0986, Mean Discriminator output=-0.0129
Epoch[1/10], Step[15/41], loss=1.0995, Mean Discriminator output=-0.0129
Epoch[1/10], Step[20/41], loss=1.0991, Mean Discriminator output=-0.0129
Epoch[1/10], Step[25/41], loss=1.0986, Mean Discriminator output=-0.0129
Epoch[1/10], Step[30/41], loss=1.0975, Mean Discrimin

# extracting filters/features

# Clustering techniques
most likely DBSCAN don't require to specify number of clusters however this can explode making difficult to use, the best option for now

# Layer Wise Relevance Propagation Or similiar techniques
The purpose of understanding CNN, see which portion of data gives more importance, mostly likely create a new CNN using trained weights from autoencoder with the final layers self-created based of the results of clustering. This can allow us to treate this CNN as supervised technique like however in reality as it is based on purely unsupervised techniques. This supervised like technique allows us to use technique like layer wise relevance propagation to understand the CNN giving us insight of the working of features learned by the cNN.